In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [15]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [16]:
X_train[:, 0].shape

(784,)

In [25]:
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    x -= np.max(x, axis=0)  # Subtract max value for numerical stability
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def forward(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = relu(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2

def relu_derivative(x):
    return x > 0

def one_hot(y):
    one_hot_Y = np.zeros((y.size, y.max() + 1))
    one_hot_Y[np.arange(y.size), y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_y = one_hot(Y)
    derivative_z2 = A2 - one_hot_y
    derivative_w2 = 1 / m * derivative_z2.dot(A1.T)
    derivative_b2 = 1 / m * np.sum(derivative_z2)
    derivative_z1 = W2.T.dot(derivative_z2) * relu_derivative(Z1)
    derivative_w1 = 1 / m * derivative_z1.dot(X.T)
    derivative_b1 = 1 / m * np.sum(derivative_z1)
    return derivative_w1, derivative_b1, derivative_w2, derivative_b2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.001, 5000)